In [1]:
import configparser
import pymongo

config = configparser.ConfigParser()
config.read("credentials.ini")

client = pymongo.MongoClient("mongodb+srv://{}:{}@cluster0-fbceu.mongodb.net/test?retryWrites=true&w=majority".format(config['MONGO']['user'], config['MONGO']['password']))
db = client.hawkerv1

In [59]:
def insert_hawker(db, name, code, address, image):
    return db['hawker'].insert_one({
        "name": name,
        "code": code,
        "address": address,
        "image": image
    })

def update_hawker(db, code, mapping):
    return db.hawker.update_one({"code": "THM"}, {"$set": mapping}).modified_count

In [60]:
name = "Tanglin Halt Market"
code = "THM"
address = "48A Tanglin Halt Rd, Singapore 148813"
image = ""
inserted_id = insert_hawker(db, name, code, address, image).inserted_id

modified = update_hawker(db, "THM", {"address": "48A Tanglin Halt Rd, Singapore 148813"})

In [70]:
def fetch_all_hawkers(db, _id=0):
    hawkers = []
    for doc in db.hawker.find({}, {"_id": _id}):
        hawkers.append(doc)
    return hawkers

def fetch_hawker_by_code(db, code, _id=0):
    return db.hawker.find_one({"code": code}, {"_id": _id})

In [62]:
fetch_hawker_by_code(db, "THM")

{'name': 'Tanglin Halt Market',
 'code': 'THM',
 'address': '48A Tanglin Halt Rd, Singapore 148813',
 'image': ''}

In [69]:
def fetch_stall_by_id(db, stallId, _id=0):
    return db.stall.find_one({"stallId": stallId}, {"_id": _id})

def fetch_stalls_by_location(db, location, _id=0):
    return [doc for doc in db.stall.find({"location": location}, {"_id": _id})]

def insert_stall(db, name, stall_type, location, stallNo, food, about, contact):
    stallId = "{}_{}".format(location, stallNo)
    if fetch_stall_by_id(db, stallId):
        raise Exception("Store already exists!")
    stall = {
        "name": name,
        "type": stall_type,
        "location": location,
        "stallNo": stallNo,
        "stallId": stallId,
        "food": food,
        "about": about,
        "contact": contact
    }
    inserted_stall = db['stall'].insert_one(stall).inserted_id
    return inserted_stall, modified_hawker

In [67]:
name = "Cafe 66"
stall_type = "Drinks"
location = "THM"
stallNo = "01-01"
stallId = "THM_01-01"
food = [
    {
        "name": "Coffee 'O'",
        "id": 1,
        "description": "Black coffee",
        "price": 0.7,
        "image": "",
        "available": True
    },
    {
        "name": "Coffee w Milk",
        "id": 2,
        "description": "",
        "price": 0.9,
        "image": "",
        "available": True
    }
]
about = {
    "recommended": "Coffe, Tea",
    "description": "Coffee stall running since 1966",
    "images": []
}
contact = {
    "poc": "",
    "number": "",
    "openingHours": ""
}
stall_id, modified = insert_stall(db, name, stall_type, location, stallNo, food, about, contact)

In [71]:
fetch_stall_by_id(db, "THM_01-01")
fetch_stalls_by_location(db, "THM")

[{'name': 'Cafe 66',
  'type': 'Drinks',
  'location': 'THM',
  'stallNo': '01-01',
  'stallId': 'THM_01-01',
  'food': [{'name': "Coffee 'O'",
    'id': 1,
    'description': 'Black coffee',
    'price': 0.7,
    'image': '',
    'available': True},
   {'name': 'Coffee w Milk',
    'id': 2,
    'description': '',
    'price': 0.9,
    'image': '',
    'available': True}],
  'about': {'recommended': 'Coffe, Tea',
   'description': 'Coffee stall running since 1966',
   'images': []},
  'contact': {'poc': '', 'number': '', 'openingHours': ''}}]

In [115]:
import datetime

today = datetime.datetime.now().strftime("%d%m%Y")
[doc for doc in db.dailyListing.find({"date": {"$gte": today}}, {"_id": 0})]

[{'name': 'Tanglin Halt Market',
  'code': 'THM',
  'address': '48A Tanglin Halt Rd, Singapore 148813',
  'image': '',
  'date': '16052020',
  'stalls': [{'stallId': 'THM_01-01',
    'name': 'Cafe 66',
    'food': [{'name': "Coffee 'O'",
      'id': 1,
      'description': 'Black coffee',
      'price': 0.7,
      'image': '',
      'available': True,
      'quantity': -1},
     {'name': 'Coffee w Milk',
      'id': 2,
      'description': '',
      'price': 0.9,
      'image': '',
      'available': True,
      'quantity': -1}]}],
  'orderAvailable': True,
  'meal': 'lunch'}]

In [144]:
def fetch_all_listings(db, _id=0):
    today = datetime.datetime.now().strftime("%d%m%Y")
    return [doc for doc in db.dailyListing.find({"date": {"$gte": today}}, {"_id": _id})]

def fetch_listing(db, date, _id=0):
    return db.dailyListing.find_one({"date": date}, {"_id": _id})

def insert_listing(db, date, code, meal, orderAvailable=True):
    if fetch_listing(db, date):
        raise Exception("Listing for this date already exists, use update instead!")
    listing = fetch_hawker_by_code(db, code)
    listing['date'] = date
    listing['stalls'] = []
    for stall in fetch_stalls_by_location(db, code):
        listing['stalls'].append({
            "stallId": stall['stallId'],
            "name": stall['name'],
            "available": True,
            "food": [{**food, 'quantity': -1} for food in stall['food'] if food['available']]
        })
    listing['orderAvailable'] = orderAvailable
    listing['meal'] = meal
    return db.dailyListing.insert_one(listing).inserted_id

def del_listing(db, date):
    return db.dailyListing.delete_one({"date": date}).deleted_count

def update_stall_availability(db, date, stallId, availability):
    listing = fetch_listing(db, date)
    if not listing:
        raise Exception("Listing unavailable for this date")
    stalls = listing['stalls']
    for stall in stalls:
        if stall['stallId'] == stallId:
            stall['available'] = availability
            updated = db.dailyListing.update_one({"date": date}, {"$set": {"stalls": stalls}}).modified_count
            return updated
    raise Exception("No such stall in this listing")
    
def update_food_quantity(db, date, stallId, foodId, quantity):
    listing = fetch_listing(db, date)
    if not listing:
        raise Exception("Listing unavailable for this date")
    stalls = listing['stalls']
    for stall in stalls:
        if stall['stallId'] == stallId:
            for food in stall['food']:
                if food['id'] == foodId:
                    food['quantity'] = quantity
                    updated = db.dailyListing.update_one({"date": date}, {"$set": {"stalls": stalls}}).modified_count
                    return updated
    raise Exception("No such stall or food in this listing")

In [145]:
update_stall_availability(db, "16052020", "THM_01-01", True)

1

In [151]:
type(1) 

int

In [148]:
update_food_quantity(db, "16052020", "THM_01-01", 1, -1)

1

In [147]:
fetch_listing(db, "16052020")

{'name': 'Tanglin Halt Market',
 'code': 'THM',
 'address': '48A Tanglin Halt Rd, Singapore 148813',
 'image': '',
 'date': '16052020',
 'stalls': [{'stallId': 'THM_01-01',
   'name': 'Cafe 66',
   'available': True,
   'food': [{'name': "Coffee 'O'",
     'id': 1,
     'description': 'Black coffee',
     'price': 0.7,
     'image': '',
     'available': True,
     'quantity': 10},
    {'name': 'Coffee w Milk',
     'id': 2,
     'description': '',
     'price': 0.9,
     'image': '',
     'available': True,
     'quantity': -1}]}],
 'orderAvailable': True,
 'meal': 'lunch'}

In [136]:
insert_listing(db, "16052020", "THM", "lunch")

In [133]:
del_listing(db, "16052020")

1